In [3]:
# 1. Uruchom kod, który wyświetli na ekranie numer wersji Sparka.

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print(spark.version)

3.1.3


In [11]:
# 2. Utwórz kolekcję danych RDD z liczbami całkowitymi z zakresu 1-20. Wyświetl na ekranie
# wszystkie elementy, ich liczbę, sumę i średnią. Utwórz i wyświetl nową kolekcję RDD z
# liczbami powyżej średniej.

# Wygeneruj kolejne liczby całkowite z zakresu 1-20 i zapisz w kolekcji RDD. Wyświetl
# strukturę na ekranie.

liczby = spark.range(1,21).rdd.map(lambda el: el.id)
print(liczby.collect())

# Oblicz i wyświetl na ekranie:

# Liczbę elementów w kolekcji RDD - skorzystaj z metody count() kolekcji RDD
ile = spark.range(1,21).rdd.count()

print(f"Liczb jest {ile}")

# Sumę elementów kolekcji RDD – skorzystaj z metody reduce() kolekcji RDD.
suma = spark.sparkContext.parallelize(range(20)).reduce(lambda a,b: a+b)
print(f"Ich suma wynosi {suma}")

# Średnią elementów w kolekcji RDD – skorzystaj z metody mean() kolekcji RDD
avg = spark.range(1,21).rdd.map(lambda el: el.id).mean()
print(f"Ich średnia wynosi {avg}")

# Utwórz nową kolekcję RDD zawierającą tylko elementy większe od średniej i wyświetl
# ją na ekranie. Skorzystaj z metody filter kolekcji RDD.
liczby = spark.range(1,21).rdd.map(lambda el: el.id).filter(lambda el: el>avg)
print("Liczby powyżej średniej:")
print(liczby.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Liczb jest 20
Ich suma wynosi 190
Ich średnia wynosi 10.5
Liczby powyżej średniej:
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [60]:
# 3. W kolejnej komórce utwórz kolekcję danych odczytaną z dołączonego pliku README.md
# Sparka. Policz ile elementów ma kolekcja. Następnie utwórz nową kolekcję ograniczoną do
# elementów zawierających słowo „Spark”, policz ile ich jest oraz wypisz na ekranie.

dane = spark.sparkContext.textFile("README.md")
print(dane.first())
print(dane.take(5))
daneSpark = dane.filter(lambda el: el.find("Spark")>-1)
print(daneSpark.count())
print(daneSpark.take(3))

# Apache Spark
['# Apache Spark', '', 'Spark is a unified analytics engine for large-scale data processing. It provides', 'high-level APIs in Scala, Java, Python, and R, and an optimized engine that', 'supports general computation graphs for data analysis. It also supports a']
19
['# Apache Spark', 'Spark is a unified analytics engine for large-scale data processing. It provides', 'rich set of higher-level tools including Spark SQL for SQL and DataFrames,']


In [61]:
# 4. Korzystając z kolekcji dane i daneSpark utworzonych w poprzednim zadaniu policz liczbę
# wszystkich znaków w każdej z tych kolekcji.

ile1 = dane.map(lambda el: len(el)).reduce(lambda a,b: a+b)
print(ile1)
ile2 = daneSpark.map(lambda el: len(el)).reduce(lambda a,b: a+b)
print(ile2)

4380
1507


In [63]:
# 5. Wygeneruj nową kolekcję ograniczając kolekcję dane z poprzednich zadań do linii
# zawierających słowo „example”. Wyświetl kolekcję na ekranie, a następnie zapisz w pliku.

daneSpark2 = dane.filter(lambda el: el.find("example")>-1)
print(daneSpark2.collect())
daneSpark2.saveAsTextFile("daneExample")

['To build Spark and its example programs, run:', 'Spark also comes with several sample programs in the `examples` directory.', 'To run one of them, use `./bin/run-example <class> [params]`. For example:', '    ./bin/run-example SparkPi', 'will run the Pi example locally.', 'You can set the MASTER environment variable when running examples to submit', 'examples to a cluster. This can be a mesos:// or spark:// URL,', 'can also use an abbreviated class name if the class is in the `examples`', '    MASTER=spark://host:7077 ./bin/run-example SparkPi', 'Many of the example programs print usage help if no params are given.']


In [6]:
# 6. Korzystając ze Sparka odczytaj dane z pliku ludzie.json dołączonego do materiałów i
# wyświetl dane z tego pliku.

ds = spark.read.json("ludzie.json")
ds.show()

+-------+----+
|   imie|wiek|
+-------+----+
| Michał|  18|
|Andrzej|  30|
| Tomasz|  19|
|   Ania|  24|
|  Basia|  32|
|  Iwona|  19|
| Monika|  22|
| Marcin|  17|
| Łukasz|  31|
+-------+----+



In [8]:
# 7. Wygeneruj nową kolekcję klasy Dataset, która będzie zawierała wyłącznie imiona osób z
# kolekcji ds. Wyświetl nową kolekcję na ekranie.

ds.createOrReplaceTempView("ds")
spark.sql("SELECT imie FROM ds").show()
print("to samo co:");
ds.select("imie").show()

+-------+
|   imie|
+-------+
| Michał|
|Andrzej|
| Tomasz|
|   Ania|
|  Basia|
|  Iwona|
| Monika|
| Marcin|
| Łukasz|
+-------+

to samo co:
+-------+
|   imie|
+-------+
| Michał|
|Andrzej|
| Tomasz|
|   Ania|
|  Basia|
|  Iwona|
| Monika|
| Marcin|
| Łukasz|
+-------+



In [23]:
# 8. Korzystając z notacji SQL Sparka wykonaj następujące zadania:

# 8.1. Wyświetl na ekranie dane osób z kolekcji ds, które mają więcej niż 21 lat.
ds.filter(ds.wiek>21).show()

# 8.2. Znajdź i wyświetl na ekranie średni wiek osób z kolekcji.
from pyspark.sql.functions import avg
ds.select(avg("wiek")).show()

# 8.3. Wyświetl na ekranie imiona rozpoczynające się na literę A.
spark.sql("SELECT imie FROM ds WHERE imie LIKE 'A%'").show()

# 8.4. Wyświetl na ekranie dane osoby najmłodszej i najstarszej.
print("Osoba najmłodsza:")
spark.sql("SELECT * FROM ds ORDER BY wiek ASC LIMIT 1").show()
print("Osoba najstarsza:")
spark.sql("SELECT * FROM ds ORDER BY wiek DESC LIMIT 1").show()

# 8.5. Oblicz średnią wieku, a następnie wyświetl na ekranie osobno dane osób, które mają
# mniej i więcej lat niż wynosi średnia.
spark.sql("SELECT AVG(wiek) FROM ds").show()
print("mniej od średniej:")
spark.sql("SELECT * FROM ds WHERE wiek<(SELECT AVG(wiek) FROM ds)").show()
print("więcej od średniej:")
spark.sql("SELECT * FROM ds WHERE wiek>(SELECT AVG(wiek) FROM ds)").show()

+-------+----+
|   imie|wiek|
+-------+----+
|Andrzej|  30|
|   Ania|  24|
|  Basia|  32|
| Monika|  22|
| Łukasz|  31|
+-------+----+

+------------------+
|         avg(wiek)|
+------------------+
|23.555555555555557|
+------------------+

+-------+
|   imie|
+-------+
|Andrzej|
|   Ania|
+-------+

Osoba najmłodsza:
+------+----+
|  imie|wiek|
+------+----+
|Marcin|  17|
+------+----+

Osoba najstarsza:
+-----+----+
| imie|wiek|
+-----+----+
|Basia|  32|
+-----+----+

+------------------+
|         avg(wiek)|
+------------------+
|23.555555555555557|
+------------------+

mniej od średniej:
+------+----+
|  imie|wiek|
+------+----+
|Michał|  18|
|Tomasz|  19|
| Iwona|  19|
|Monika|  22|
|Marcin|  17|
+------+----+

więcej od średniej:
+-------+----+
|   imie|wiek|
+-------+----+
|Andrzej|  30|
|   Ania|  24|
|  Basia|  32|
| Łukasz|  31|
+-------+----+



In [5]:
# 9. Wczytaj plik data.data, którego używaliśmy na lab03 do kolekcji RDD złożonej z obiektów
# Osoba. Definicja klasy Osoba znajduje się w pliku Osoba.py. Sprawdź czy kolekcja została
# utworzona poprawnie wyświetlając liczbę elementów oraz zawartość pierwszy z nich.

class Osoba:
    def __init__(self, dane):
        self.wiek = int(dane[0]);
        self.formaZatrudnienia = dane[1];
        self.wagaKontrolna = dane[2];
        self.wyksztalcenie = dane[3];
        self.wyksztalcenieLiczbowo = int( dane[4]);
        self.stanCywilny = dane[5];
        self.grupaZawodowa = dane[6];
        self.relacjaRodzinna = dane[7];
        self.rasa = dane[8];
        self.plec = dane[9];
        self.zysk = dane[10];
        self.strata = dane[11];
        self.godzinyTygodniowo = int( dane[12]);
        self.kraj = dane[13];
        self.klasaZysku = dane[14];
    
    def __str__(self):
        return str(vars(self))

    def __repr__(self):
        return "{"+str(self.wiek) + ", " + str(self.kraj) + ", " +str(self.wyksztalcenie) + "...}"

lines = spark.sparkContext.textFile("data.data").filter(lambda l: len(l)>0 and l.find("?")<0)
dane = lines.map(lambda line: Osoba(line.split(", ")))
#sprawdzenie
print(dane.count())
print(dane.first())

7940


23/05/21 21:09:55 ERROR Executor: Exception in task 0.0 in stage 3.0 (TID 5)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    bytes = self.serializer.dumps(vs)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 427, in dumps
    return pickle.dumps(obj, pickle_protocol)
_pickle.PicklingError: Can't pickle <class '__main__.Osoba'>: attribute lookup Osoba on __main__ failed

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:652)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(Pyt

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 5) (jupyter.ksi.local executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    bytes = self.serializer.dumps(vs)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 427, in dumps
    return pickle.dumps(obj, pickle_protocol)
_pickle.PicklingError: Can't pickle <class '__main__.Osoba'>: attribute lookup Osoba on __main__ failed

	at org.apache.spark.api.python.BaseP

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58558)
Traceback (most recent call last):
  File "/usr/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.9/socketserver.py", line 720, in __init__
    self.handle()
  File "/opt/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/opt/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/opt/spark/python/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile)
  File "/opt/spark/python/pyspark/serializers.py", line 564, in read_int
    r

In [73]:
# 10. W kolejnych komórkach dodaj kod umożliwiający uzyskanie informacji opisanych poniżej.
# Wyniki porównaj z wynikami z lab03.

# 10.1. Lista kobiet pochodzących z Meksyku z wykształceniem HS-grad.
d = dane.filter(lambda el: el.plec=="Female" and el.wyksztalcenie=="HS-grad")
print(d)

# 10.2. Liczba osób, które wzięły udział w spisie z podziałem na kobiety i mężczyzn.
d = dane.groupBy("plec").map(lambda gr:(gr[0],len(gr[1])))
print(d)

# 10.3. Średni wiek osób Never-married o formie zatrudnienia Private.
d = dane.filter(lambda el: el.stanCywilny=="Never-married" and el.formaZatrudnienia=="Private").map(lambda el: el.wiek)
print(d.sum()/d.count())

# 10.4. Liczba osób o wykształceniu liczbowo od 9 do 13 pracujących od +20 do 30 godzin tygodniowo pogrupowana według form zatrudnienia.
d = dane.groupBy("formaZatrudnienia").filter(lambda x: x.wyksztalcenieLiczbowo>=9 and x.wyksztalcenieLiczbowo<=13 and x.godzinyTygodniowo>=20 and x.godzinyTygodniowo<=30).map(lambda gr:(gr[0],len(gr[1])))
print(d)

# 10.5. Liczby osób z USA z takim samym wykształceniem.
d = dane.groupBy("wyksztalcenie").filter(lambda el: el.kraj=="United-States").map(lambda gr:(gr[0],len(gr[1])))
print(d)

# 10.6. Średni wiek osób z klasy zysku ”>50K” w każdej grupie zawodowej.
d = dane.filter(lambda el: el.stanCywilny=="klasaZysku").groupBy("grupaZawodowa").map(lambda el: el.wiek)
print(d)

PythonRDD[682] at RDD at PythonRDD.scala:53
PythonRDD[687] at RDD at PythonRDD.scala:53
27.633520249221185
PythonRDD[694] at RDD at PythonRDD.scala:53
PythonRDD[699] at RDD at PythonRDD.scala:53
PythonRDD[704] at RDD at PythonRDD.scala:53
